In [4]:
%pylab inline
import os; os.environ['KERAS_BACKEND'] = 'tensorflow'
import numpy as np
from sklearn.preprocessing import OneHotEncoder


Populating the interactive namespace from numpy and matplotlib


In [5]:
class CharTransformer:
    def __init__(self):
        pass
    
    def build_vocab(self, text):
        self.vocab = sorted(list(set(text)))
        self.vocab_size = len(self.vocab)
    
    def encode_one_hot(self, char):
        k = self.vocab.index(char)
        arr = np.zeros(self.vocab_size, dtype=np.bool8)
        arr[k] = 1
        return arr
    
    def decode_one_hot(self, arr):
        idx = np.argmax(arr)
        return self.vocab[idx]
    
    def batch_encode_one_hot(self, text):
        return np.array([self.encode_one_hot(c) for c in text])

In [6]:
def rnn_sliding_window_indices(seq_len, time_steps, batch_size, epochs=1):
    from collections import deque

    i = 0
    assert seq_len >= time_steps + batch_size + 2, "sequence must be larger than time_steps+batch_size+2"
    while True:
        if i > seq_len - time_steps - batch_size - 2:
            if epochs <= 0:
                break
            epochs -= 1
            i = 0        
        batch = []
        for j in range(batch_size):
            x_start = i + j
            x_stop = x_start + time_steps
            y_start = x_start + 1
            y_stop = x_stop + 1
            batch.append((x_start, x_stop, y_start, y_stop))
        yield batch
        i += 1

def rnn_data_gen(seq, time_steps=None, batch_size=None, epochs=None, last_label_only=True):
    windows = rnn_sliding_window_indices(seq.shape[0], 
                                         time_steps=time_steps,
                                         batch_size=batch_size,
                                         epochs=epochs)
    for batch_indices in windows:
        x_batch = []
        y_batch = []
        for x_start, x_stop, y_start, y_stop in batch_indices:
            x_batch.append(seq[x_start:x_stop])
            if last_label_only:
                y_batch.append(seq[y_stop-1])
            else:
                y_batch.append(seq[y_start:y_stop])
        yield (np.array(x_batch), np.array(y_batch))

In [7]:
# Quotes from Rick from Rick and Morty

TEXT = """
Ohh yea, you gotta get schwifty.
He's not pressing charges... That's gotta be the "you shot me" equivalent of not being mad.
Get off the high road Summer. We all got pink eye because you wouldn't stop texting on the toilet.
He's a real piece of shit.
I'm not looking for judgement, just a yes or no. Can you assimilate a giraffe?
We need a hang glider, and a crotch less uncle sam costume, and I want the entire field of your largest stadium covered end to end with naked red heads, and I want the stands packed with every man that remotely resembles my father.
Yea... If you spend all day shuffling words around you can make anything sound bad... Morty..
Like nothing shady ever happened in a fully furnished office? You ever hear about Wall Street Morty? You know what those guys do in their fancy board rooms? They take their balls and dip 'em in cocaine and wipe 'em all over each other. You know Grandpa goes around and he does his business in public because grandpa isn't shady.
There is no god, in your face! One dot muthafucka!
Not today bitch!
That guy is the Red Grin Grumbold of pretending he knows what's going on... Oh you agree huh? You like that Red Grin Grumbold reference? Well guess what, I made him up. You really are your father's children. Think for yourselves, don't be sheep.
Yo! What up my glip glops!
Then it's time to get your beak wet tonight playa!
It's like the N word and the C word had a baby, and it was raised by all the bad words for Jews.
Don't break an arm jerking yourself off Morty.
25 shmeckles? I-I-I-I don't even know what that- what is that? Is that a lot?
ALLAH BLEHHHH AKBAR
Me irresponsible?? All I wanted was for you to hand me a screwdriver! But instead you had me buckle down and...make you a...roofie...juice serum, so you can roofie that poor girl at your school. W-w-w-w--are you kidding me, Morty?! You're really gonna try to take the high road on this one?? Y'know your-you're a little creep, Morty! Your-your-your-you're just a little creepy creep person!!
I don't like it here Morty. I can't abide bureaucracy. I don't like being told where to go and what to do. I consider it a violation. Did you get those seeds all the way up your butt?
"I don't think we can perform our new song, The Recipe For Concentrated Dark Matter for a crowd this tiny!"
I'll be with Reuben in my workshop while you guys are having another day in Phil Collin's proverbial paradise.
"I'll tell you how I feel about school, Jerry: it's a waste of time. Bunch of people runnin' around bumpin' into each other, got a guy up front says, '2 + 2,' and the people in the back say, '4.' Then the bell rings and they give you a carton of milk and a piece of paper that says you can go take a dump or somethin'. I mean, it's not a place for smart people, Jerry. I know that's not a popular opinion, but that's my two cents on the issue."
Monument to Compromise.
I mixed in some Praying Mantis DNA. You know a Praying Mantis is the exact opposite of a Vole, Morty? They mate once and then bluergh cut each other's heads off.
I'm sorry, Morty. It's a bummer. In reality you're as dumb as they come.
I'm sorry Summer, your opinion means very little to me.
ISN'T IT OBVIOUS MORTY? I FROZE HIM.
Its a device Morty, that when you put it in your ear, you can enter people's dreams Morty. Its just like that movie that you keep crowing about.
It's a figure of speech, Morty! They're bureaucrats! I don't respect them. Just keep shooting, Morty! You have no idea what prison is like here!
Its fine, everythings is fine. theres an infinite number of realities morty, and in a few dozens of those i got lucky and turned everything back to normal
Its not like we can do this every week, we get 3 or 4 more of these tops.
Lemme hear everybody say hey-yo! All the ladies say yeaaah! Everybody over thirty do this with your hand! Everybody with a red shirt jump up and down! Yo, everyone whose first name begins with an L who isn't hispanic, walk in a circle the same number of times as the square root of your age times teeenn!
Listen, Morty, I hate to break it to you but what people call "love" is just a chemical reaction that compels animals to breed.
Puffy vagina
Morty, can you get to the left nipple?
Not for the men they cheat on.
Sometimes science is a lot more art, than science. A lot of people don't get that.
That's Right Morty! This is gonna be a lot like that. Except you know. Its gonna~Belch~ make sense.
There is no god, Summer; gotta rip that band-aid off now you'll thank me later.
They'll just fall right out of my ass! I've done this too many times!
They're not infinite universes left in sync with the show,
They're robots Morty! It's okay to shoot them! They're just robots!
WUBBA LUBBA DUB DUBS!!!
Oh, I'm sorry Morty, are you the scientist or are you the kid who wanted to get laid?
THOSE GUYS ARE INSIDE YOU BUILDING A PIECE OF SHIT ETHAN!! THEY'RE INSIDE YOU BUILDING A MONUMENT TO COMPROMISE!! fuck them... fuck those people... fuck this whole thing ETHAN.
This isn't Game of Thrones, Morty
We don't whitewash it either, Morty. I mean, the pirates are really rapey.
Well, she's my daughter, Summer. I outrank you. Or, family means nothing, in which case, don't play that card.
We're gonna nine eleven this bitch unless Morty gets better math grades!
Wha, me irresponsible ?! All I wanted you to do was to hand me a screwdriver, Morty !
What about the reality where Hitler cured cancer, Morty? The answer is: Don't think about it.
What are you looking at, mother fucker!
What just happened back there?" "Why don't you ask the smartest people in the universe, Jerry? Oh yeah belch you can't. They blew up.
Whoa, spoilers! I'm a whole season behind.
Wow I really crononberg'd up the whole place huh Morty, just a bunch a cronenbergs walkin around
You ask alotta questions Morty, not very charismatic of you
You don't have to try to impress me Morty.
You gotta shove these seeds way up your butt morty, waay up there
You have to turn them on Morty! The shoes need to be turned on!
You know what a vole is, Morty? YOU KNOW WHAT A VOLE IS?
You're growing up fast, Morty. You're going into a great big thorn straight into my ass...
You're our boy dog, don't even trip
You're young, you have your whole life ahead of you, and your anal cavity is still taut yet malleable.
You're not gonna believe this, because it usually never happens, but I made a mistake
Oh I say good sir, oh harumph, ohh a bope-a-dope-o-bobo.
Do you know how many characters there are in the Simpsons Morty? There's like a-a BILLION CHARACTERS, M-Morty. There was an episode where Former President BUSH was their neighbor!
GRAAAAAASSSSSSS.... tastes bad.
No jumping in the sewer!
Wubbalubbadubdub!
What is this, 90's Conan?
Lick, lickity, lick my balls!
"""

charxform = CharTransformer()
charxform.build_vocab(TEXT)
x_one_hot = charxform.batch_encode_one_hot(TEXT)

In [8]:
BATCH_SIZE = 64
TIME_STEPS = 20
INPUT_SIZE = charxform.vocab_size
OUTPUT_SIZE = INPUT_SIZE
NB_SAMPLES = len(TEXT) - TIME_STEPS - BATCH_SIZE - 2

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

gen = rnn_data_gen(x_one_hot, time_steps=TIME_STEPS, batch_size=BATCH_SIZE, epochs=2000)
opt = Adam()
early_stop_cb = EarlyStopping(monitor='loss', patience=0, verbose=0, mode='auto')
model_ckpt_cb = ModelCheckpoint("model.ckpt", monitor='loss', verbose=0, save_best_only=True, mode='auto')


model = Sequential()
model.add(LSTM(128, batch_input_shape=(BATCH_SIZE, TIME_STEPS, INPUT_SIZE), return_sequences=True))
model.add(LSTM(OUTPUT_SIZE, activation="softmax"))
model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(gen, samples_per_epoch=NB_SAMPLES , nb_epoch=16, verbose=2, callbacks=[model_ckpt_cb, early_stop_cb])

Using TensorFlow backend.


Epoch 1/16


In [ ]:
def sample(probs, temperature):
    # helper function to sample an index from a probability array
    preds = np.asarray(probs).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def sample_chars(seed, model, max_length=100):
    text = seed
    while len(text) < max_length:
        x = np.zeros((BATCH_SIZE, TIME_STEPS, INPUT_SIZE), dtype=np.float32)
        x[0] = charxform.batch_encode_one_hot(text[-TIME_STEPS:])
        probs = model.predict_proba(x, verbose=0)[0]
        index = sample(probs, max(0.2, np.random.random() - .5))
        text += charxform.vocab[index]
    print text
    


model.load_weights("model.ckpt")
sample_chars("seedlings seedlings seedlings", model)
